### Sentiment Analysis of Johnson & Johnson's Janssen vaccine tweets UK

**Original Author:** Elena Stamatelou.<br/>
**Additional Info:** Sentiment analysis on streaming twitter data using Spark Structured Streaming & Python. https://github.com/stamatelou/twitter_sentiment_analysis<br/>
**Last Modified:**  

In [1]:
# Import the os module 
import os

# Set the PYSPARK_SUBMIT_ARGS to the appropriate spark-sql-kafka package
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'

In [2]:
# Install textblob for the sentiment analysis
import sys
!{sys.executable} -m pip install -U textblob

Requirement already up-to-date: textblob in /opt/conda/lib/python3.8/site-packages (0.15.3)


In [3]:
# Import Sparksession and sql functions 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob


In [4]:
# text classification

# Define methods from TextBlob
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity

def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity

def sentiment_detection(value):
    if value < 0: 
        return 'Negative'
    elif value > 0: 
        return 'Positive'
    else:
        return 'Neutral'

# polarity detection
# Define as user defined fuction to embed method in the spark environment 
polarity_detection_udf = udf(polarity_detection, StringType())

# subjectivity detection
# Define as user defined fuction to embed method in the spark environment 
subjectivity_detection_udf = udf(subjectivity_detection, StringType())

# sentiment detection
# Define as user defined fuction to embed method in the spark environment 
sentiment_detection_udf = udf(sentiment_detection, StringType())

In [5]:
# Import the findspark module 
import findspark

# Initialize via the full spark path
findspark.init("/usr/local/spark/")

In [6]:
# create Spark session
spark = SparkSession.builder \
   .master("local[*]") \
   .appName("TwitterSentAnalysis") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()

In [7]:
try: 
    # Read Tweets from the Kafka topic vaccine 
    tweet_df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
        .option("subscribe", "johnsonandjohnson_UK") \
        .option("startingOffsets", "latest") \
        .load()
except: 
    print("Unexpected error:", sys.exc_info()[0])

In [8]:
try: 
    # Cast the data into a json
    tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING) as json_data")
    
    # extract the tweet and user info
    text_user = tweet_df_string.select(json_tuple('json_data', 'created_at','text', 'user').alias('created_at', 'text', 'json_user')) 
    
    # extract screen_name and location from user info
    text_user_info = text_user.select('text', 'created_at', json_tuple('json_user', 'location').alias('location')) 
    
    # preprocessing
    text_user_info = text_user_info.na.replace('', 'None')
    text_user_info = text_user_info.na.drop()
    
    text_user_info = text_user_info.withColumn('text', F.regexp_replace('text', r'http\S+', ''))
    text_user_info = text_user_info.withColumn('text', F.regexp_replace('text', '@\w+', ''))
    text_user_info = text_user_info.withColumn('text', F.regexp_replace('text', '#', ''))
    text_user_info = text_user_info.withColumn('text', F.regexp_replace('text', 'RT', ''))
    text_user_info = text_user_info.withColumn('text', F.regexp_replace('text', ':', ''))

    # polarity detection
    # Append polarity to dataframe
    text_user_info = text_user_info.withColumn("polarity", polarity_detection_udf(text_user_info.text))
    
    # subjectivity detection
    # Append subjectivity to dataframe 
    text_user_info = text_user_info.withColumn("subjectivity", subjectivity_detection_udf(text_user_info.text))
    
    #sentiment detection
    # Append sentiment to dataframe
    text_user_info = text_user_info.withColumn("sentiment", sentiment_detection_udf(text_user_info.polarity))
    
    # repartition 'Returns a new DataFrame partitioned by the given partitioning expressions. The resulting DataFrame is hash partitioned.'
    text_user_info = text_user_info.repartition(1)
    
    # country filter 
    text_user_info = text_user_info.filter("location == 'UK' OR location LIKE 'UK %' OR location LIKE '% UK' OR location LIKE '% UK %' \
                  OR location = 'GB' OR location LIKE 'GB %' OR location LIKE '% GB' OR location LIKE '% GB %' \
                  OR location = 'uk' OR location LIKE 'uk %' OR location LIKE '% uk' OR location LIKE '% uk %' \
                  OR location = 'UNITED KINGDOM' OR location LIKE 'UNITED KINGDOM %' OR location LIKE '% UNITED KINGDOM' OR location LIKE '% UNITED KINGDOM %' \
                  OR location = 'United Kingdom' OR location LIKE 'United Kingdom %' OR location LIKE '% United Kingdom' OR location LIKE '% United Kingdom %' \
                  OR location = 'gb' OR location LIKE 'gb %' OR location LIKE '% gb' OR location LIKE '% gb %' \
                  OR location = 'united kingdom' OR location LIKE 'united kingdom %' OR location LIKE '% united kingdom' OR location LIKE '% united kingdom %' \
                  OR location = 'Britain' OR location LIKE 'Britain %' OR location LIKE '% Britain' OR location LIKE '% Britain %' \
                  OR location = 'United Kingdom of Great Britain and Northern Ireland' OR location LIKE 'United Kingdom of Great Britain and Northern Ireland %' OR location LIKE '% United Kingdom of Great Britain and Northern Ireland' OR location LIKE '% United Kingdom of Great Britain and Northern Ireland %' \
                  OR location = 'United Kingdom of Great Britain' OR location LIKE 'United Kingdom of Great Britain %' OR location LIKE '% United Kingdom of Great Britain' OR location LIKE '% United Kingdom of Great Britain %' \
                  OR location = 'Great Britain' OR location LIKE 'Great Britain %' OR location LIKE '% Great Britain' OR location LIKE '% Great Britain %' \
                  OR location = 'Northern Ireland' OR location LIKE 'Northern Ireland %' OR location LIKE '% Northern Ireland' OR location LIKE '% Northern Ireland %' \
                  OR location = 'northern ireland' OR location LIKE 'northern ireland %' OR location LIKE '% northern ireland' OR location LIKE '% northern ireland %' \
                  OR location = 'NIR' OR location LIKE 'NIR %' OR location LIKE '% NIR' OR location LIKE '% NIR %' \
                  OR location = 'England' OR location LIKE 'England %' OR location LIKE '% England' OR location LIKE '% England %' \
                  OR location = 'england' OR location LIKE 'england %' OR location LIKE '% england' OR location LIKE '% england %' \
                  OR location = 'ENG' OR location LIKE 'ENG %' OR location LIKE '% ENG' OR location LIKE '% ENG %' \
                  OR location = 'Scotland' OR location LIKE 'Scotland %' OR location LIKE '% Scotland' OR location LIKE '% Scotland %' \
                  OR location = 'scotland' OR location LIKE 'scotland %' OR location LIKE '% scotland' OR location LIKE '% scotland %' \
                  OR location = 'SCT' OR location LIKE 'SCT %' OR location LIKE '% SCT' OR location LIKE '% SCT %' \
                  OR location = 'Wales' OR location LIKE 'Wales %' OR location LIKE '% Wales' OR location LIKE '% Wales %' \
                  OR location = 'wales' OR location LIKE 'wales %' OR location LIKE '% wales' OR location LIKE '% wales %' \
                  OR location = 'Bedfordshire' OR location LIKE 'Bedfordshire %' OR location LIKE '% Bedfordshire' OR location LIKE '% Bedfordshire %' \
                  OR location = 'Berkshire' OR location LIKE 'Berkshire %' OR location LIKE '% Berkshire' OR location LIKE '% Berkshire %' \
                  OR location = 'Buckinghamshire' OR location LIKE 'Buckinghamshire %' OR location LIKE '% Buckinghamshire' OR location LIKE '% Buckinghamshire %' \
                  OR location = 'Cambridgeshire' OR location LIKE 'Cambridgeshire %' OR location LIKE '% Cambridgeshire' OR location LIKE '% Cambridgeshire %' \
                  OR location = 'Cheshire' OR location LIKE 'Cheshire %' OR location LIKE '% Cheshire' OR location LIKE '% Cheshire %' \
                  OR location = 'Cornwall' OR location LIKE 'Cornwall %' OR location LIKE '% Cornwall' OR location LIKE '% Cornwall %' \
                  OR location = 'Cumberland' OR location LIKE 'Cumberland %' OR location LIKE '% Cumberland' OR location LIKE '% Cumberland %' \
                  OR location = 'Derbyshire' OR location LIKE 'Derbyshire %' OR location LIKE '% Derbyshire' OR location LIKE '% Derbyshire %' \
                  OR location = 'Devon' OR location LIKE 'Devon %' OR location LIKE '% Devon' OR location LIKE '% Devon %' \
                  OR location = 'Dorset' OR location LIKE 'Dorset %' OR location LIKE '% Dorset' OR location LIKE '% Dorset %' \
                  OR location = 'Durham' OR location LIKE 'Durham %' OR location LIKE '% Durham' OR location LIKE '% Durham %' \
                  OR location = 'Essex' OR location LIKE 'Essex %' OR location LIKE '% Essex' OR location LIKE '% Essex %' \
                  OR location = 'Gloucestershire' OR location LIKE 'Gloucestershire %' OR location LIKE '% Gloucestershire' OR location LIKE '% Gloucestershire %' \
                  OR location = 'Hampshire' OR location LIKE 'Hampshire %' OR location LIKE '% Hampshire' OR location LIKE '% Hampshire %' \
                  OR location = 'Herefordshire' OR location LIKE 'Herefordshire %' OR location LIKE '% Herefordshire' OR location LIKE '% Herefordshire %' \
                  OR location = 'Hertfordshire' OR location LIKE 'Hertfordshire %' OR location LIKE '% Hertfordshire' OR location LIKE '% Hertfordshire %' \
                  OR location = 'Huntingdonshire' OR location LIKE 'Huntingdonshire %' OR location LIKE '% Huntingdonshire' OR location LIKE '% Huntingdonshire %' \
                  OR location = 'Kent' OR location LIKE 'Kent %' OR location LIKE '% Kent' OR location LIKE '% Kent %' \
                  OR location = 'Lancashire' OR location LIKE 'Lancashire %' OR location LIKE '% Lancashire' OR location LIKE '% Lancashire %' \
                  OR location = 'Leicestershire' OR location LIKE 'Leicestershire %' OR location LIKE '% Leicestershire' OR location LIKE '% Leicestershire %' \
                  OR location = 'Lincolnshire' OR location LIKE 'Lincolnshire %' OR location LIKE '% Lincolnshire' OR location LIKE '% Lincolnshire %' \
                  OR location = 'Middlesex' OR location LIKE 'Middlesex %' OR location LIKE '% Middlesex' OR location LIKE '% Middlesex %' \
                  OR location = 'Norfolk' OR location LIKE 'Norfolk %' OR location LIKE '% Norfolk' OR location LIKE '% Norfolk %' \
                  OR location = 'Northamptonshire' OR location LIKE 'Northamptonshire %' OR location LIKE '% Northamptonshire' OR location LIKE '% Northamptonshire %' \
                  OR location = 'Northumberland' OR location LIKE 'Northumberland %' OR location LIKE '% Northumberland' OR location LIKE '% Northumberland %' \
                  OR location = 'Nottinghamshire' OR location LIKE 'Nottinghamshire %' OR location LIKE '% Nottinghamshire' OR location LIKE '% Nottinghamshire %' \
                  OR location = 'Oxfordshire' OR location LIKE 'Oxfordshire %' OR location LIKE '% Oxfordshire' OR location LIKE '% Oxfordshire %' \
                  OR location = 'Rutland' OR location LIKE 'Rutland %' OR location LIKE '% Rutland' OR location LIKE '% Rutland %' \
                  OR location = 'Shropshire' OR location LIKE 'Shropshire %' OR location LIKE '% Shropshire' OR location LIKE '% Shropshire %' \
                  OR location = 'Somerset' OR location LIKE 'Somerset %' OR location LIKE '% Somerset' OR location LIKE '% Somerset %' \
                  OR location = 'Staffordshire' OR location LIKE 'Staffordshire %' OR location LIKE '% Staffordshire' OR location LIKE '% Staffordshire %' \
                  OR location = 'Suffolk' OR location LIKE 'Suffolk %' OR location LIKE '% Suffolk' OR location LIKE '% Suffolk %' \
                  OR location = 'Surrey' OR location LIKE 'Surrey %' OR location LIKE '% Surrey' OR location LIKE '% Surrey %' \
                  OR location = 'Sussex' OR location LIKE 'Sussex %' OR location LIKE '% Sussex' OR location LIKE '% Sussex %' \
                  OR location = 'Warwickshire' OR location LIKE 'Warwickshire %' OR location LIKE '% Warwickshire' OR location LIKE '% Warwickshire %' \
                  OR location = 'Westmorland' OR location LIKE 'Westmorland %' OR location LIKE '% Westmorland' OR location LIKE '% Westmorland %' \
                  OR location = 'Wiltshire' OR location LIKE 'Wiltshire %' OR location LIKE '% Wiltshire' OR location LIKE '% Wiltshire %' \
                  OR location = 'Worcestershire' OR location LIKE 'Worcestershire %' OR location LIKE '% Worcestershire' OR location LIKE '% Worcestershire %' \
                  OR location = 'Aberdeenshire' OR location LIKE 'Aberdeenshire %' OR location LIKE '% Aberdeenshire' OR location LIKE '% Aberdeenshire %' \
                  OR location = 'Angus' OR location LIKE 'Angus %' OR location LIKE '% Angus' OR location LIKE '% Angus %' \
                  OR location = 'Argyllshire' OR location LIKE 'Argyllshire %' OR location LIKE '% Argyllshire' OR location LIKE '% Argyllshire %' \
                  OR location = 'Ayrshire' OR location LIKE 'Ayrshire %' OR location LIKE '% Ayrshire' OR location LIKE '% Ayrshire %' \
                  OR location = 'Banffshire' OR location LIKE 'Banffshire %' OR location LIKE '% Banffshire' OR location LIKE '% Banffshire %' \
                  OR location = 'Berwickshire' OR location LIKE 'Berwickshire %' OR location LIKE '% Berwickshire' OR location LIKE '% Berwickshire %' \
                  OR location = 'Bute' OR location LIKE 'Bute %' OR location LIKE '% Bute' OR location LIKE '% Bute %' \
                  OR location = 'Caithness' OR location LIKE 'Caithness %' OR location LIKE '% Caithness' OR location LIKE '% Caithness %' \
                  OR location = 'Clackmannanshire' OR location LIKE 'Clackmannanshire %' OR location LIKE '% Clackmannanshire' OR location LIKE '% Clackmannanshire %' \
                  OR location = 'Dumfriesshire' OR location LIKE 'Dumfriesshire %' OR location LIKE '% Dumfriesshire' OR location LIKE '% Dumfriesshire %' \
                  OR location = 'Dunbartonshire' OR location LIKE 'Dunbartonshire %' OR location LIKE '% Dunbartonshire' OR location LIKE '% Dunbartonshire %' \
                  OR location = 'East Lothian' OR location LIKE 'East Lothian %' OR location LIKE '% East Lothian' OR location LIKE '% East Lothian %' \
                  OR location = 'Edinburghshire' OR location LIKE 'Edinburghshire %' OR location LIKE '% Edinburghshire' OR location LIKE '% Edinburghshire %' \
                  OR location = 'Elginshire' OR location LIKE 'Elginshire %' OR location LIKE '% Elginshire' OR location LIKE '% Elginshire %' \
                  OR location = 'Fife' OR location LIKE 'Fife %' OR location LIKE '% Fife' OR location LIKE '% Fife %' \
                  OR location = 'Forfarshire' OR location LIKE 'Forfarshire %' OR location LIKE '% Forfarshire' OR location LIKE '% Forfarshire %' \
                  OR location = 'Haddingtonshire' OR location LIKE 'Haddingtonshire %' OR location LIKE '% Haddingtonshire' OR location LIKE '% Haddingtonshire %' \
                  OR location = 'Inverness-shire' OR location LIKE 'Inverness-shire %' OR location LIKE '% Inverness-shire' OR location LIKE '% Inverness-shire %' \
                  OR location = 'Kincardineshire' OR location LIKE 'Kincardineshire %' OR location LIKE '% Kincardineshire' OR location LIKE '% Kincardineshire %' \
                  OR location = 'Kinross-shire ' OR location LIKE 'Kinross-shire %' OR location LIKE '% Kinross-shire' OR location LIKE '% Kinross-shire %' \
                  OR location = 'Kirkcudbrightshire' OR location LIKE 'Kirkcudbrightshire %' OR location LIKE '% Kirkcudbrightshire' OR location LIKE '% Kirkcudbrightshire %' \
                  OR location = 'Lanarkshire' OR location LIKE 'Lanarkshire %' OR location LIKE '% Lanarkshire' OR location LIKE '% Lanarkshire %' \
                  OR location = 'Midlothian ' OR location LIKE 'Midlothian %' OR location LIKE '% Midlothian' OR location LIKE '% Midlothian %' \
                  OR location = 'Moray' OR location LIKE 'Moray %' OR location LIKE '% Moray' OR location LIKE '% Moray %' \
                  OR location = 'Nairnshire' OR location LIKE 'Nairnshire %' OR location LIKE '% Nairnshire' OR location LIKE '% Nairnshire %' \
                  OR location = 'Orkney Islands' OR location LIKE 'Orkney Islands %' OR location LIKE '% Orkney Islands' OR location LIKE '% Orkney Islands %' \
                  OR location = 'Peeblesshire' OR location LIKE 'Peeblesshire %' OR location LIKE '% Peeblesshire' OR location LIKE '% Peeblesshire %' \
                  OR location = 'Perthshire' OR location LIKE 'Perthshire %' OR location LIKE '% Perthshire' OR location LIKE '% Perthshire %' \
                  OR location = 'Renfrewshire' OR location LIKE 'Renfrewshire %' OR location LIKE '% Renfrewshire' OR location LIKE '% Renfrewshire %' \
                  OR location = 'Ross & Cromarty' OR location LIKE 'Ross & Cromarty %' OR location LIKE '% Ross & Cromarty' OR location LIKE '% Ross & Cromarty %' \
                  OR location = 'Roxburghshire' OR location LIKE 'Roxburghshire %' OR location LIKE '% Roxburghshire' OR location LIKE '% Roxburghshire %' \
                  OR location = 'Selkirkshire' OR location LIKE 'Selkirkshire %' OR location LIKE '% Selkirkshire' OR location LIKE '% Selkirkshire %' \
                  OR location = 'Shetland' OR location LIKE 'Shetland %' OR location LIKE '% Shetland' OR location LIKE '% Shetland %' \
                  OR location = 'Sutherland' OR location LIKE 'Sutherland %' OR location LIKE '% Sutherland' OR location LIKE '% Sutherland %' \
                  OR location = 'West Lothian' OR location LIKE 'West Lothian %' OR location LIKE '% West Lothian' OR location LIKE '% West Lothian %' \
                  OR location = 'Wigtonshire' OR location LIKE 'Wigtonshire %' OR location LIKE '% Wigtonshire' OR location LIKE '% Wigtonshire %' \
                  OR location = 'Anglesey' OR location LIKE 'Anglesey %' OR location LIKE '% Anglesey' OR location LIKE '% Anglesey %' \
                  OR location = 'Breconshire' OR location LIKE 'Breconshire %' OR location LIKE '% Breconshire' OR location LIKE '% Breconshire %' \
                  OR location = 'Caernarfonshire' OR location LIKE 'Caernarfonshire %' OR location LIKE '% Caernarfonshire' OR location LIKE '% Caernarfonshire %' \
                  OR location = 'Cardiganshire' OR location LIKE 'Cardiganshire %' OR location LIKE '% Cardiganshire' OR location LIKE '% Cardiganshire %' \
                  OR location = 'Carmarthenshire' OR location LIKE 'Carmarthenshire %' OR location LIKE '% Carmarthenshire' OR location LIKE '% Carmarthenshire %' \
                  OR location = 'Denbighshire' OR location LIKE 'Denbighshire %' OR location LIKE '% Denbighshire' OR location LIKE '% Denbighshire %' \
                  OR location = 'Flintshire' OR location LIKE 'Flintshire %' OR location LIKE '% Flintshire' OR location LIKE '% Flintshire %' \
                  OR location = 'Glamorgan' OR location LIKE 'Glamorgan %' OR location LIKE '% Glamorgan' OR location LIKE '% Glamorgan %' \
                  OR location = 'Merionethshire' OR location LIKE 'Merionethshire %' OR location LIKE '% Merionethshire' OR location LIKE '% Merionethshire %' \
                  OR location = 'Monmouth' OR location LIKE 'Monmouth %' OR location LIKE '% Monmouth' OR location LIKE '% Monmouth %' \
                  OR location = 'Montgomeryshire' OR location LIKE 'Montgomeryshire %' OR location LIKE '% Montgomeryshire' OR location LIKE '% Montgomeryshire %' \
                  OR location = 'Pembrokeshire' OR location LIKE 'Pembrokeshire %' OR location LIKE '% Pembrokeshire' OR location LIKE '% Pembrokeshire %' \
                  OR location = 'Radnorshire' OR location LIKE 'Radnorshire %' OR location LIKE '% Radnorshire' OR location LIKE '% Radnorshire %' \
                  OR location = 'Antrim' OR location LIKE 'Antrim %' OR location LIKE '% Antrim' OR location LIKE '% Antrim %' \
                  OR location = 'Armagh' OR location LIKE 'Armagh %' OR location LIKE '% Armagh' OR location LIKE '% Armagh %' \
                  OR location = 'Down' OR location LIKE 'Down %' OR location LIKE '% Down' OR location LIKE '% Down %' \
                  OR location = 'Fermanagh' OR location LIKE 'Fermanagh %' OR location LIKE '% Fermanagh' OR location LIKE '% Fermanagh %' \
                  OR location = 'Pembrokeshire' OR location LIKE 'Pembrokeshire %' OR location LIKE '% Pembrokeshire' OR location LIKE '% Pembrokeshire %' \
                  OR location = 'Londonderry' OR location LIKE 'Londonderry %' OR location LIKE '% Londonderry' OR location LIKE '% Londonderry %' \
                  OR location = 'Tyrone' OR location LIKE 'Tyrone %' OR location LIKE '% Tyrone' OR location LIKE '% Tyrone %' \
                  OR location = 'London' OR location LIKE 'London %' OR location LIKE '% London' OR location LIKE '% London %' \
                  OR location = 'london' OR location LIKE 'london %' OR location LIKE '% london' OR location LIKE '% london %' \
                  OR location = 'Manchester' OR location LIKE 'Manchester %' OR location LIKE '% Manchester' OR location LIKE '% Manchester %' \
                  OR location = 'Birmingham' OR location LIKE 'Birmingham %' OR location LIKE '% Birmingham' OR location LIKE '% Birmingham %'")
    
    #option("header", "True"). \
    #Write the spark stream
    writeTweet = text_user_info.writeStream. \
        format("csv"). \
        option("checkpointLocation", "./storage_johnson&johnson_UK/"). \
        option("path", "./storage_johnson&johnson_UK/"). \
        outputMode("append"). \
        queryName("UK_jnj_tweets"). \
        trigger(processingTime='50 seconds'). \
        start()
    
    print("----- streaming is running -------")
    
    writeTweet.awaitTermination()
    
except: 
    print("Unexpected error:", sys.exc_info())

----- streaming is running -------
Unexpected error: (<class 'KeyboardInterrupt'>, KeyboardInterrupt(), <traceback object at 0x7f919148f6c0>)


In [17]:
spark.stop()

In [ ]:
#scotland = ["Aberdeenshire", "Angus", "Argyllshire", "Ayrshire", "Banffshire", "Berwickshire", Bute, Caithness, Clackmannanshire, Dumfriesshire, East Lothian,
#                                           Edinburghshire,  Elginshire, Fife, Forfarshire, Haddingtonshire, Inverness-shire, Kincardineshire, Kinross-shire, Kirkcudbrightshire,
#                                           Lanarkshire, Linlithgowshire, Midlothian, Moray, Nairnshire, Orkney Islands, Peeblesshire, Perthshire, Renfrewshire, Ross & Cromarty,
#                                           Roxburghshire, Selkirkshire, Shetland, Stirlingshire, Sutherland, West Lothian, Wigtonshire ]
#wales = [Anglesey, Breconshire, Caernarfonshire, Cardiganshire, Carmarthenshire, Denbighshire, Flintshire, Glamorgan, Merionethshire, Monmouth, Montgomeryshire, Pembrokeshire, Radnorshire]
#northern_ireland = [ Antrim, Armagh, Down, Fermanagh, Londonderry, Tyrone]